In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [3]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 

### User Input

In [4]:
hot_songs = pd.read_csv('hot_songs.csv')
hot_songs

,song_title,artist(s)
0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenz..."
1,Do We Have A Problem?,Nicki Minaj X Lil Baby
2,Easy On Me,Adele
3,Heat Waves,Glass Animals
4,Stay,The Kid LAROI & Justin Bieber
...,...,...
95,Iffy,Chris Brown
96,When I'm Gone,Alesso / Katy Perry
97,Fair Trade,Drake Featuring Travis Scott
98,Megan's Piano,Megan Thee Stallion


In [5]:
# User input and checking if the song is in the hot list.

song_input = input("Please enter a name of a song: ").title()

if song_input in hot_songs.values:
    print()
    print("Your song is in the hot songs list!")
    print("Here is another song you could listen: ")
    print()
    print(hot_songs.sample())
#else:
    #print("Please, check if you have any typos in the song name. If not, unfortunately, the song is not in the hot list.")

Please enter a name of a song: moth to a flame

Your song is in the hot songs list!
Here is another song you could listen: 

  song_title      artist(s)
8      Ghost  Justin Bieber


### Getting the features of the input song

In [6]:
input_song = sp.search(q = song_input, limit = 1, market = "GB")

In [7]:
input_song

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Moth+To+A+Flame&type=track&market=GB&offset=0&limit=1',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1h6Cn3P4NGzXbaXidqURXs'},
       'href': 'https://api.spotify.com/v1/artists/1h6Cn3P4NGzXbaXidqURXs',
       'id': '1h6Cn3P4NGzXbaXidqURXs',
       'name': 'Swedish House Mafia',
       'type': 'artist',
       'uri': 'spotify:artist:1h6Cn3P4NGzXbaXidqURXs'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
       'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
       'id': '1Xyo4u8uXC1ZmMpatF05PJ',
       'name': 'The Weeknd',
       'type': 'artist',
       'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/3udIvIqyBG1RrHxXrUZDoK'},
     'href': 'https://api.spotify.com/v1/albums/3udIvIqyBG1RrHxXrUZDoK',
     'id':

In [2]:
input_song_features = sp.audio_features(input_song["tracks"]["items"][0]["id"] )

NameError: name 'sp' is not defined

In [9]:
input_song_features = pd.DataFrame(input_song_features)
input_song_features_num = input_song_features._get_numeric_data()


In [1]:
input_song_features_num = input_song_features_num.drop(columns = ["key", "mode", "time_signature"])

NameError: name 'input_song_features_num' is not defined

In [11]:
input_song_features_num

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.542,0.659,-7.289,0.0389,0.00279,0,0.105,0.109,120.122,234000


In [12]:
model = load("scaler.pickle")

In [13]:
input_song_scaled = model.transform(input_song_features_num)

In [14]:
input_song_scaled = pd.DataFrame(input_song_scaled, columns = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness",
                          "valence", "tempo", "duration_ms"])

In [15]:
model_kmeans = load("kmeans_9.pickle")

In [16]:
input_song_cluster = model_kmeans.predict(input_song_scaled)

In [17]:
input_song_cluster

array([5], dtype=int32)

In [18]:
model_kmeans

KMeans(n_clusters=9, random_state=1234)